In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision import transforms as T
import torch.nn.functional as F
import cv2

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_dir='../input/projet-deep-learning/train'
test_dir='../input/projet-deep-learning/test'
annotation_dir="../input/projet-deep-learning/annotations_train.csv"

# Loading and preparing the data

In [ ]:
annotations=pd.read_csv(annotation_dir)

making the labels have 0 or 1 value

In [ ]:
columns=annotations.columns.difference(['id'])
for c in columns:
  annotations[c]=annotations[c]-1

In [ ]:
upcolors=['upblack','upwhite','upred','uppurple','upyellow','upgray','upblue','upgreen'] 
downcolors=['downblack','downwhite','downpink','downpurple','downyellow','downgray','downblue','downgreen','downbrown']

In [ ]:
annotations["upmulticolor"]=0
for up in upcolors:
  annotations["upmulticolor"]=annotations["upmulticolor"]+annotations[up]
annotations["upmulticolor"]=(annotations["upmulticolor"]==0)*1
annotations["downmulticolor"]=0
for down in downcolors:
  annotations["downmulticolor"]=annotations["downmulticolor"]+annotations[down]
annotations["downmulticolor"]=(annotations["downmulticolor"]==0)*1

applying one hot encoding to the age

In [ ]:
one_hot_columns=['age']
annotations=pd.get_dummies(annotations,columns=one_hot_columns)

reorganizing the data

In [ ]:
columns=['id','gender', 'hair','backpack', 'bag', 'clothes', 'handbag', 'hat', 'up','down', 
'downblack', 'downblue', 'downbrown', 'downgray', 'downgreen','downpink', 'downpurple', 'downwhite', 'downyellow','downmulticolor',
'upblack', 'upblue', 'upgray','upgreen', 'uppurple', 'upred', 'upwhite', 'upyellow','upmulticolor',
'age_0', 'age_1', 'age_2', 'age_3']
annotations=annotations[columns]

In [ ]:
class MyDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.annotations =csv_file
        self.root_dir = root_dir
        self.transform = transform
        l=[]
        for filename in os.listdir(root_dir):
          l.append(filename)
        self.l=l
    def __len__(self):
        return len(self.l)

    def __getitem__(self, idx):
        l=self.l
        img_name = os.path.join(self.root_dir,
                                l[idx])
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        id_img=int(l[idx].split("_")[0])
        ann=self.annotations
        labels = np.array(ann[ann.id==id_img].iloc[0,1:])
        #sample = {'image': image, 'labels': labels}

        if self.transform:
            image = self.transform(image)

        return (image, labels)

In [ ]:
class MytestDataset(Dataset):

    def __init__(self, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        l=[]
        for filename in os.listdir(root_dir):
          l.append(filename)
        self.l=l
    def __len__(self):
        return len(self.l)

    def __getitem__(self, idx):
        l=self.l
        img_name = os.path.join(self.root_dir,
                                l[idx])
        id=l[idx]
        image = io.imread(img_name)
        if self.transform:
            sample = self.transform(image)

        return [id,sample]

In [ ]:
def get_data(train_dir,test_dir,batch_size):
    data_transforms = {
      'train': T.Compose([
          T.ToTensor(),
          T.RandomHorizontalFlip(p=0.5),
          T.RandomRotation(10),
          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
      ]),
      'val': T.Compose([
          T.ToTensor(),
          T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
      ]),
    }
    # Load data
    full_training_data = MyDataset(annotations,train_dir, transform=data_transforms['train'])
    test_data =  MytestDataset(test_dir, transform=data_transforms['val'])


    # Create train and validation splits
    num_samples = len(full_training_data)
    training_samples = int(num_samples*0.7+1)
    validation_samples = num_samples - training_samples

    training_data, validation_data = torch.utils.data.random_split(full_training_data, [training_samples, validation_samples])

    # Initialize dataloaders
    train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True, num_workers=1)
    val_loader = torch.utils.data.DataLoader(validation_data, batch_size, shuffle=False, num_workers=1)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size, shuffle=False, num_workers=1)
    return train_loader, val_loader, test_loader

# Building model functions


In [ ]:
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3,3), stride= 1, padding=1)
    torch.nn.init.xavier_uniform_(self.conv1.weight)
    self.conv2 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3,3), stride= 1, padding=1)
    torch.nn.init.xavier_uniform_(self.conv2.weight)
    self.conv3 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), stride= 1, padding=1)
    torch.nn.init.xavier_uniform_(self.conv3.weight)
    self.conv4 = torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), stride= 1, padding=1)
    torch.nn.init.xavier_uniform_(self.conv4.weight)
    self.conv5 = torch.nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3), stride= 1, padding=1)
    torch.nn.init.xavier_uniform_(self.conv5.weight)
    self.conv6 = torch.nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3,3), stride= 1, padding=1)
    torch.nn.init.xavier_uniform_(self.conv6.weight)
    self.conv7 = torch.nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=(3,3), stride= 1, padding=1)
    torch.nn.init.xavier_uniform_(self.conv7.weight)
    self.fc1 = torch.nn.Linear(in_features=1*1*1024, out_features=1024)
    self.fc2 = torch.nn.Linear(in_features=1024, out_features=32)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x= F.max_pool2d(x, kernel_size=2, stride=2)

    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2)
    
    x = self.conv3(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2)

    x = self.conv4(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2)

    x = self.conv5(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2)

    x = self.conv6(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2)

    x = self.conv7(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=1, stride=2)
  #flatten the feature maps into long vector
    x = x.view(x.shape[0], -1)

    x = self.fc1(x)
    #x = F.relu(x)

    x = self.fc2(x)

    return x

In [ ]:
def get_cost_function():
  cost_function = torch.nn.BCEWithLogitsLoss()
  return cost_function

In [ ]:
def get_optimizer(net, lr, wd, momentum):
  optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=wd)
  return optimizer

# Training and testing the model

In [ ]:
def test(net, data_loader, cost_function, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  net.eval() # Strictly needed if network contains layers which has different behaviours between train and test
  with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(data_loader):
      # Load data into GPU
      inputs = inputs.to(device)
      targets = targets.to(device)
        
      # Forward pass
      outputs = net(inputs)

      # Apply the loss
      targets=targets.float()
      loss = cost_function(outputs,targets)
      # Better print something
      samples+=inputs.shape[0]
      print(samples)
      cumulative_loss += loss.item()
      targets_1=targets.cpu().detach().numpy()
      predictions=outputs.cpu().detach().numpy()

      up_p=predictions[:,9:19]
      down_p=predictions[:,19:28]
      age_p=predictions[:,28:]
      up_z = np.zeros_like(predictions[:,9:19])
      down_z = np.zeros_like(predictions[:,19:28])
      age_z=np.zeros_like(predictions[:,28:])
      up_z[np.arange(len(up_p)), up_p.argmax(1)] = 1
      down_z[np.arange(len(down_p)), down_p.argmax(1)] = 1
      age_z[np.arange(len(age_p)), age_p.argmax(1)] = 1
      predictions[:,9:19]=up_z
      predictions[:,19:28]=down_z
      predictions[:,28:]=age_z
      
      predictions=(predictions>0)*1
      cumulative_accuracy+=((predictions == targets_1)*1).sum()/predictions.shape[1]

  return cumulative_loss/samples, cumulative_accuracy/samples*100

In [ ]:
def train(net,data_loader,optimizer,cost_function, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  
  net.train() # Strictly needed if network contains layers which has different behaviours between train and test
  for batch_idx, (inputs, targets) in enumerate(data_loader):
    # Load data into GPU
    inputs = inputs.to(device)
    targets = targets.to(device)
    # Forward pass
    outputs = net(inputs)
    # Apply the loss
    targets=targets.float()
    loss = cost_function(outputs,targets)
    # Backward pass
    loss.backward()
    # Update parameters
    optimizer.step()
    # Reset the optimizer
    optimizer.zero_grad()

    # Better print something, no?
    samples+=inputs.shape[0]
    print(samples)
    cumulative_loss += loss.item()
    targets_1=targets.cpu().detach().numpy()
    predictions=outputs.cpu().detach().numpy()
    up_p=predictions[:,9:19]
    down_p=predictions[:,19:28]
    age_p=predictions[:,28:]
    up_z = np.zeros_like(predictions[:,9:19])
    down_z = np.zeros_like(predictions[:,19:28])
    age_z=np.zeros_like(predictions[:,28:])
    up_z[np.arange(len(up_p)), up_p.argmax(1)] = 1
    down_z[np.arange(len(down_p)), down_p.argmax(1)] = 1
    age_z[np.arange(len(age_p)), age_p.argmax(1)] = 1
    predictions[:,9:19]=up_z
    predictions[:,19:28]=down_z
    predictions[:,28:]=age_z
    predictions=(predictions>0)*1
    cumulative_accuracy+=((predictions == targets_1)*1).sum()/predictions.shape[1]
  return cumulative_loss/samples, cumulative_accuracy/samples*100

In [ ]:
batch_size=128
device='cuda:0' 
learning_rate=0.0005
weight_decay=0.000001
momentum=0.9
epochs=20
  
train_loader, val_loader, test_loader = get_data(train_dir,test_dir,batch_size)
  
net = CNN().to(device)
  
optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)

cost_function = get_cost_function()
print('start training')
train_accuracy_per_e=[]
val_accuracy_per_e=[]
for e in range(epochs):
  train_loss, train_accuracy = train(net, train_loader, optimizer, cost_function)
  val_loss, val_accuracy = test(net, val_loader, cost_function)
  train_accuracy_per_e.append(train_accuracy)
  val_accuracy_per_e.append(val_accuracy)
  print('Epoch: {:d}'.format(e+1))
  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  print('-----------------------------------------------------')

print('After training:')
train_loss, train_accuracy = test(net, train_loader, cost_function)
print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
val_loss, val_accuracy = test(net, val_loader, cost_function)
print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
print('-----------------------------------------------------')

In [ ]:
epochs_l=[e+1 for e in range(epochs)]
plt.figure()
plt.plot(epochs_l,train_accuracy_per_e,label='Train')
plt.plot(epochs_l,val_accuracy_per_e,label='Validation')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend()
plt.show()


In [ ]:
plt.savefig("plt.png")

In [ ]:
torch.save(net.state_dict(), "CNN_label_prediction")

# Predicting test data


In [ ]:
net = CNN().to(device)
net.load_state_dict(torch.load("CNN_label_prediction"), strict=True)

In [ ]:
def predict(net, test_data,device='cuda:0'):
  l=[]
  p=[]
  samples=0
  net.eval()
  with torch.no_grad():
    for batch_idx, (ids, inputs) in enumerate(test_data):
      # Load data into GPU
      inputs = inputs.to(device)
      # Forward pass
      outputs = net(inputs)
      samples+=inputs.shape[0]
      print(samples)
      predictions=np.array(outputs.cpu())
      up_p=predictions[:,9:19]
      down_p=predictions[:,19:28]
      age_p=predictions[:,28:]
      up_z = np.zeros_like(predictions[:,9:19])
      down_z = np.zeros_like(predictions[:,19:28])
      age_z=np.zeros_like(predictions[:,28:])
      up_z[np.arange(len(up_p)), up_p.argmax(1)] = 1
      down_z[np.arange(len(down_p)), down_p.argmax(1)] = 1
      age_z[np.arange(len(age_p)), age_p.argmax(1)] = 1
      predictions[:,9:19]=up_z
      predictions[:,19:28]=down_z
      predictions[:,28:]=age_z
      predictions=(predictions>0)*1
      p.append(predictions)
      l.append(ids)
    return l,p


In [ ]:
predictions=predict(net, test_loader,device='cuda:0')

In [ ]:
sub=pd.DataFrame()
for i in range(len(predictions[1])):
  for j in range(predictions[1][i].shape[0]):
    s= pd.DataFrame( predictions[1][i][j].reshape((1,32)) , columns=columns[1:] )
    s['id']=predictions[0][i][j]
    sub=sub.append(s)

In [ ]:
sub['age']=sub['age_0'] * 0+sub['age_1']*1+sub['age_2']*2+sub['age_3']*3
annotations=pd.read_csv(annotation_dir)
sub=sub[annotations.columns]
for c in sub.columns.difference(['id']):
  sub[c]=sub[c]+1

In [ ]:
sub.to_csv('submission.csv')

In [ ]:
sub.head(10)